<h1>Preliminaries</h1>
<p>This codebase seeks to replicate the following paper:</p>
<p><strong>Radioactive data: tracing through training</strong><br />
    <a href="https://arxiv.org/pdf/2002.00937.pdf">https://arxiv.org/pdf/2002.00937.pdf</a>
        </p>
    
<p>We have done a full refactor of the original open source Facebook code. Separately we also created a basic example demonstrating the optimization of a resnet18 cifar10 classifier. This is used as a template for the second task below.</p>
<p>This notebook goes through the following steps:</p>
   <ol>
       <li>Mark a certain subset of the CIFAR10 dataset (target data) using a resnet18 pretrained on imagenet (marking network)</li>
       <li>Use the modified CIFAR10 dataset to train a new resnet18 (target network)</li>
       <li>Attempt to detect radioactivity in the target network</li>
    </ol>
</p>
<p><strong>Note:</strong><br/>
In our example the marking network is pretrained on imagenet, while our target data is CIFAR10.
According to section 5.5 of the paper, even a marking network trained on a different
data distribution will output radioactive markings that are useful when applied to at least 10% of a class. 
This number could vary, we are just quoting the minimum radioactive data percentage shown in the paper. </p>
<p>If the marking network was trained on the same data distribution it is likely that a lower percentage of the
   target data would require marking to achieve the same p value in the detection stage.</p>
<p>When performing our testing we found that a 10% marking only produced a p-value of around 0.85, indicating that we should go higher. We are currently performing a range of tests on this.</p>

<h1>Creating Radioactive Data (Image Marking)</h1>

<p>This is self-contained within <strong>make_data_radioactive.py</strong></p>
<p>The <em>"if __name__ == '__main__':"</em> will:</p>
<ol>
    <li>Create an experiments directory <em>experiments/radioactive</em>. All further paths are relative to this directory (except tensorboard).</li>
    <li>Setup logging to <em>marking.log</em></li>
    <li>Randomly select an image class from the CIFAR10 dataset, then randomly sample a certain percentage of images in that class. After marking, these images are saved with their original index in the filename, allowing us to create a merged training set when training our classifier further on. These unmarked images are sent to TensorBoard for viewing.</li>
    <li>Download a pretrained resnet18 for use as the marking network.
    <li>Random carriers are created for all classes. These are saved to <em>carrier.pth</em> for use when detecting radioactivity.</li>
    <li>Call the main function that performs the training.</li>
</ol>
<p>Unlike a classifier, we are backpropagating all the way to the images (similar to a GAN) from a special loss function designed to maximize carrier alignment, while minimizing feature and image difference between the current and original. See section 3.3 of the paper for more details.</p>
<p>Given the above, the main method runs through batches of images, performing a random augmentation then running Adam over a desired number of epochs using this special loss function to add a subtle mark to each image</p>
<p>After each epoch we clamp all pixels so they don't drift outside of 0-255.</p>
<p>Once all epochs are completed for a given batch of images, the newly marked images are saved into <em>marked_images/{class_id}.</em></p>
<p>Each batch of images has it's own TensorBoard log for you to inspect the training loss over time.</p>
<p>It takes about 1:30 to train 500 images on a 980ti.</p>

In [ ]:
%run make_data_radioactive.py

<p>Once this is done, inspect the TensorBoard to examine the loss for each batch and view both the original randomly selected images and their marked versions. Tensorboard logs are saved in the <em>runs</em> directory.</p>
<p>Simply run "tensorboard --logdir runs" from the repo root, and navigate to <a target="_blank" href="http://localhost:6006">http://localhost:6006</a></p>

<h1>Training A Model</h1>
<p>This is self-contained within <strong>train_marked_classifier.py</strong></p>

<p>It is pre-configured with the relative directory paths from the marking phase.</p>

<p>In this phase we take our marked CIFAR10 images from the previous step and merge them into the CIFAR10 training set using a dataloader wrapper. We then use this merged dataset to train a fresh resnet18.</p>

<p>The only difference between this and our normal resnet18 CIFAR10 example is we created a custom class <em>MergedDataset</em> that takes a standard dataset, merging dataset and a list of indexes indicating which original examples the merge dataset indexes should replace</p>

<p>The code is set to train for 60 epochs, taking about 15 minutes on a 980ti. We originally ran to 150 epochs, but overfitting starts to occur around 60. While you wait, detailed training information will show up in the TensorBoard under the <em>trained_marked_classifier</em> tag.</p>

In [ ]:
%run train_marked_classifier.py

<h1>Detecting if a model is radioactive</h1>
<p>This is self-contained within <strong>detect_radioactivity.py</strong></p>
<p>Here we check for carrier alignment using cosine similarity</p>

<p>On the output, you should obtain a line with "log10(p)=...", which gives the (log of the) p-value of radioactivity detection averaged across all classes.</p>

In [ ]:
%run detect_radioactivity.py